In [2]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import pandas as pd
import re

# Cargar el archivo CSV
input_csv = "C:/Users/jpcan/Downloads/Inmuebles_Disponibles_para_Arrendamiento_con_barrios.csv"  # Reemplaza con la ruta de tu archivo CSV
df = pd.read_csv(input_csv)

# Filtrar solo las direcciones que mencionan "Bogotá"
df_bogota = df[df['Ciudad'].str.contains('Bogotá', case=False, na=False)]

# Función para corregir el formato de las direcciones
def correct_address_format(address):
    # Remplazar "CL" y "KR" por "Calle" y "Carrera"
    address = re.sub(r'\bCL\b', 'Calle', address, flags=re.IGNORECASE)
    address = re.sub(r'\bKR\b', 'Carrera', address, flags=re.IGNORECASE)
    
    # Unir la letra (A, B, C, D) con el número anterior si está inmediatamente después
    address = re.sub(r'(\d+)\s*([A-D])\b', r'\1\2', address)
    
    # Colocar un espacio y un "#" después de "Calle" o "Carrera"
    address = re.sub(r'(Calle|Carrera)\s*(\S+)', r'\1 \2 #', address, flags=re.IGNORECASE)
    
    # Unir los números tipo "57 60" en "57-60"
    address = re.sub(r'(\d+)\s+(\d+)', r'\1-\2', address)
    
    # Eliminar los caracteres adicionales después de los números excepto A, B, C o D
    address = re.sub(r'(\d+-\d+)(\s*[A-D]?)\s.*', r'\1\2', address)
    
    return address

# Aplicar la corrección a las direcciones
df_bogota['Direccion_Corregida'] = df_bogota['Direccion'].apply(correct_address_format)

# Imprimir las direcciones corregidas para supervisión
print(df_bogota[['Direccion', 'Direccion_Corregida']])

# Aquí es donde luego podrías seguir con el proceso de obtener el barrio y generar el nuevo CSV




                                              Direccion  \
0                                  CL 119 A 57 60 GJ 95   
1                           CL 119 A 57 60 TR 4 AP 1212   
2                           CR 58 119 A 98 TR 6 AP 1216   
8     CL 166 8H 56 AP 504 IN 2 CON TORRES DEL FERROC...   
10                          CL 27 5 78 ANTES CL 27 5 80   
...                                                 ...   
1362  KR 19         12       84     LC 2 233   CE SA...   
1363            CL 115 48 80 AP 503 DIRECCION CATASTRAL   
1366              CL 152     54    39    IN 4   AP 1114   
1367      KR 48 174B 67 IN 3 AP 310 DIRECCION CATASTRAL   
1368  AP 502 ED PRADOS NORTE 3 CL 130 9 45 O DG 129 ...   

                                    Direccion_Corregida  
0                                    Calle 119A # 57-60  
1                                    Calle 119A # 57-60  
2                                            CR 58-119A  
8     Calle 166 # 8H 56 AP 504 IN 2 CON TORRES DEL F...  
1

C:\Users\jpcan\AppData\Local\Temp\ipykernel_13652\520509836.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bogota['Direccion_Corregida'] = df_bogota['Direccion'].apply(correct_address_format)


In [24]:
import pandas as pd
import re

# Cargar el archivo CSV
input_csv = "C:/Users/jpcan/Downloads/Inmuebles_Disponibles_para_Arrendamiento_con_barrios.csv"  # Reemplaza con la ruta de tu archivo CSV
df = pd.read_csv(input_csv)

# Filtrar solo las direcciones que mencionan "Bogotá"
df_bogota = df[df['Ciudad'].str.contains('Bogotá', case=False, na=False)]

# Función para corregir el formato de las direcciones
def correct_address_format(address):
    # Remplazar "CL" y "KR" por "Calle" y "Carrera"
    address = re.sub(r'\bCL\b', 'Calle', address, flags=re.IGNORECASE)
    address = re.sub(r'\bKR\b', 'Carrera', address, flags=re.IGNORECASE)
    
    # Unir la letra (A, B, C, D) con el número anterior si está inmediatamente después
    address = re.sub(r'(\d+)\s*([A-D])\b', r'\1\2', address)
    
    # Colocar un espacio y un "#" después de "Calle" o "Carrera"
    address = re.sub(r'(Calle|Carrera)\s*(\S+)', r'\1 \2 #', address, flags=re.IGNORECASE)
    
    # Unir los números tipo "57 60" en "57-60"
    address = re.sub(r'(\d+)\s+(\d+)', r'\1-\2', address)
    
    # Eliminar los caracteres adicionales después de los números excepto A, B, C o D
    address = re.sub(r'(\d+-\d+)(\s*[A-D]?)\s.*', r'\1\2', address)
    
    # Añadir ", Bogotá, Colombia" al final de la dirección
    address += ", Bogotá, Colombia"
    
    return address

# Aplicar la corrección a las direcciones
df_bogota['Direccion_Corregida'] = df_bogota['Direccion'].apply(correct_address_format)

# Imprimir las direcciones corregidas para supervisión
print(df_bogota[['Direccion', 'Direccion_Corregida']])

# Aquí es donde luego podrías seguir con el proceso de obtener el barrio y generar el nuevo CSV


# Aquí es donde luego podrías seguir con el proceso de obtener el barrio y generar el nuevo CSV




import pandas as pd
from geopy.geocoders import Nominatim

# Inicializa el geocodificador
geolocator = Nominatim(user_agent="geoapiExercises")

# Función para obtener el barrio de una dirección
def get_neighborhood(address):
    try:
        location = geolocator.geocode(address)
        if location:
            location_detail = geolocator.reverse((location.latitude, location.longitude), exactly_one=True)
            if 'suburb' in location_detail.raw['address']:
                return location_detail.raw['address']['suburb']
            elif 'neighbourhood' in location_detail.raw['address']:
                return location_detail.raw['address']['neighbourhood']
            elif 'borough' in location_detail.raw['address']:
                return location_detail.raw['address']['borough']
        return 'Barrio no encontrado'
    except Exception as e:
        return 'Error'


# Crear la nueva columna 'Barrio'
df_bogota['Barrio'] = df_bogota['Direccion_Corregida'].apply(get_neighborhood)

# Guardar el DataFrame modificado a un nuevo archivo CSV
output_csv = "C:/Users/jpcan/Downloads/Inmuebles_Disponibles_para_Arrendamiento_con_barrios_cinco.csv"  # Especifica la ruta y nombre para el nuevo archivo CSV
df_bogota.to_csv(output_csv, index=False)

print(f"Archivo guardado como {output_csv}")



C:\Users\jpcan\AppData\Local\Temp\ipykernel_13652\574691353.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bogota['Direccion_Corregida'] = df_bogota['Direccion'].apply(correct_address_format)


                                              Direccion  \
0                                  CL 119 A 57 60 GJ 95   
1                           CL 119 A 57 60 TR 4 AP 1212   
2                           CR 58 119 A 98 TR 6 AP 1216   
8     CL 166 8H 56 AP 504 IN 2 CON TORRES DEL FERROC...   
10                          CL 27 5 78 ANTES CL 27 5 80   
...                                                 ...   
1362  KR 19         12       84     LC 2 233   CE SA...   
1363            CL 115 48 80 AP 503 DIRECCION CATASTRAL   
1366              CL 152     54    39    IN 4   AP 1114   
1367      KR 48 174B 67 IN 3 AP 310 DIRECCION CATASTRAL   
1368  AP 502 ED PRADOS NORTE 3 CL 130 9 45 O DG 129 ...   

                                    Direccion_Corregida  
0                  Calle 119A # 57-60, Bogotá, Colombia  
1                  Calle 119A # 57-60, Bogotá, Colombia  
2                          CR 58-119A, Bogotá, Colombia  
8     Calle 166 # 8H 56 AP 504 IN 2 CON TORRES DEL F...  
1

C:\Users\jpcan\AppData\Local\Temp\ipykernel_13652\574691353.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bogota['Barrio'] = df_bogota['Direccion_Corregida'].apply(get_neighborhood)
